<h2>Exploratory Analysis<h2>
   <br>
Analyze according to year and store

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
import gc

In [ ]:
# Python program to get average of a list 
def Average(lst): 
    return sum(lst) / len(lst) 


In [ ]:
#get normalized 
def Normalize(item,lst):
    
    maxmin = max(lst) - min(lst)
    return (item - Average(lst))/maxmin if maxmin >= 1 else 1
    
    

In [ ]:
from holidays_jp import CountryHolidays
# get japanese holidays in 2016.

jpHolidays = CountryHolidays.between('JP', 2010,2020)

holidays = pd.DataFrame(jpHolidays,columns=['date','name'])

In [ ]:
total_demand = []
years = []
num_store = []
demand_per_store = []
month_summary = []
weekly_summary = []
holiday_summary = []
workday_summary = []
weekend_summary = []

In [ ]:
loop = 0
#for i in range(2016,2020):
#print(loop)
i = 2019
for i in range(2010,2020):
    
    fname = str(i)
    #get years 
    years.append(i)

    #Load data
    temp = pd.read_hdf('~/work/Data/prepared_data2/key'+fname+'.h5', 'df')
    #temp = temp.drop(['id','saturation','color'])
    #get total demand 
    total_demand.append(temp['quantity'].sum())

    #print(len(temp['storeId'].unique()))

    #get number of stores
    num_store.append(len(temp['storeId'].unique()))

    #print(num_store[loop])
    #print(num_store)

    #mean normalization
    normalized_num_store = Normalize(num_store[loop],num_store)

    #print(num_store[i],normalized_num_store)

    dps = total_demand[loop]/normalized_num_store
    demand_per_store.append(dps)

    #calculate monthly demand for each year
    temp['date'] = pd.to_datetime(temp['date'])   
    temp['month'] = pd.DatetimeIndex(temp['date']).month

    months = temp.groupby("month")
    t1 = months['quantity'].sum()    
    month_summary.append(t1)
    del [months]
    del [t1]


    #Demand per Store by Day of Week

    #calculate weekly demand for each year
    temp['date'] = pd.to_datetime(temp['date'])   
    temp['day'] = pd.DatetimeIndex(temp['date']).day_name()
    days = temp.groupby("day")
    t1 = days['quantity'].sum()
    weekly_summary.append(t1)
    del [days]
    del [t1]


    #get weekends 
    temp['isWeekend']= temp.day.isin(['Saturday','Sunday'])
    #temp['isHoliday'] = temp.date.isin(holidays.date)
    weekendsdf = temp[temp['isWeekend'] == True]
    weekends = weekendsdf['date']
    #holidaysn = holidays['date']
    #holidaysn.append(weekends)
    weekendsdf= temp.groupby("day")
    t1 = weekendsdf['quantity'].sum()
    weekend_summary.append(t1)
    del [weekendsdf]
    del [t1]

    #holidays
    holidaysn = holidays['date']
    holidaysn.append(weekends)
    holidaysdf = temp[temp['isWeekend'] == True]
    #holidaysn = holidays['date']
    #holidaysn.append(weekends)
    holidaysdf= holidaysdf.groupby("day")
    t1 = holidaysdf['quantity'].sum()
    holiday_summary.append(t1)
    del [holidaysdf]
    del [t1]



    #for weekdays
    temp2 = temp[temp.date.isin(holidaysn) == False]
    temp2 = temp2.groupby("day")
    t2 = temp2['quantity'].sum() 
    workday_summary.append(t2)
    del [temp2]
    del [t2]


    loop = loop +1

    #garbage collection

    del [temp]
    del [dps]
    gc.collect()
    

In [ ]:
gc.collect()

In [ ]:
print(total_demand)
print(years)
print(num_store)
print(demand_per_store)
print(month_summary) 

<h2>Total demand by Year</h2>

In [ ]:
ax = sns.barplot(x=years, y=total_demand)
ax.set_title('Total Demand By Year')
ax.set_xlabel("years")
ax.set_ylabel("normalized demands")

<h2>Total Stores by Year</h2>

In [ ]:
ax = sns.barplot(x=years, y=num_store)
ax.set_title('Total Stores By Year')
ax.set_xlabel("years")
ax.set_ylabel("number of stores")

<h2>Demand per Store by Year<h2>

In [ ]:
ax = sns.barplot(x=years, y=demand_per_store)
ax.set_title('Demand per Store By Year')
ax.set_xlabel("years")
ax.set_ylabel("demand per Store")

<h2>Demand per Stores by Month of Year</h2>

In [ ]:
#calculate monthly average demand for 10 years
total_demand_by_month = [0,0,0,0,0,0,0,0,0,0,0,0]
for i in range(len(years)):
    for j in month_summary[i].index:
        total_demand_by_month[j-1] += month_summary[i][j]

for i in range(len(total_demand_by_month)):
    total_demand_by_month[i] = total_demand_by_month[i]/len(years)

In [ ]:
demand_per_store = []
for i in range(len(years)):
    
    temp = total_demand_by_month[i]/Normalize(num_store[i],num_store)
    demand_per_store.append(temp)
    
ax = sns.barplot(x=month_summary[0].index, y=total_demand_by_month)
ax.set_title('Demand per Store By Month of Year')
ax.set_xlabel("month")
ax.set_ylabel("demand per Store")

In [ ]:
#https://databricks.com/notebooks/recitibikenycdraft/exploratory-analysis.html


<h2>Stores Geo</h2>

In [ ]:
stores = pd.read_csv("~/work/Data/stores.csv")
stores.head()

In [ ]:
prefectures = stores.groupby("prefecture")
p_size = prefectures.size()
type(p_size)
p_size.values
#p_size.index

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.axis('equal')
ax.pie(p_size.values, labels = p_size.index)
plt.show()


In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(p_size.index,p_size.values)
plt.show()

In [ ]:
#print(p_size.values)
sns.set(style="whitegrid")
#ax = sns.barplot(x=p_size.index, y=p_size.values)
plt.figure(figsize=(20,20))

df = pd.DataFrame({'Pref':p_size.index, 'count':p_size.values/100})
print(df)


In [ ]:
chart = sns.countplot(
    data= df,
    x= 'Pref',
    palette='Set1'
)
chart.set_xticklabels(chart.get_xticklabels(), rotation=45)

In [ ]:
f, ax = plt.subplots(figsize=(7, 10))
sns.countplot(y=p_size.index, data=p_size, color="pink");

In [ ]:
sns.catplot(x="month", y="quantity", data=df[len(df)-4]);
sns.catplot(x="month", y="quantity", data=df[len(df)-3]);
sns.catplot(x="month", y="quantity", data=df[len(df)-2]);
sns.catplot(x="month", y="quantity", data=df[len(df)-4]);

In [ ]:

sns.catplot(x="month", y="quantity",jitter=False, data=df[6]);

In [ ]:
df[6][df[6].month == "2"]

In [ ]:
df[6].head(20000)

In [ ]:
month_summary

In [ ]:
num_store

In [ ]:
fname = "2010"
temp = pd.read_hdf('~/work/Data/prepared_data/key'+fname+'.h5', 'df')

In [ ]:
stemp = temp.groupby("storeId").size()

In [ ]:

stemp.index

In [ ]:
stemp.values

In [ ]:
#Demand per Store for Holidays

temp






In [ ]:


t2 = day['quantity'].sum()   

#month_summary.append(t1)

In [ ]:


#holidays.drop(['name'],axis=1)


temp['isWeekend']= temp.day.isin(['Saturday','Sunday'])


temp['isHoliday'] = temp.date.isin(holidays.date)



In [ ]:
weekends = temp[temp['isWeekend'] == True]
weekends = weekends['date']
holidaysn = holidays['date']

holidaysn.append(weekends)


In [ ]:
#for holidays
temp2 = temp[temp.isHoliday == True]
t2 = temp2['quantity'].sum() 

In [ ]:
#for weekends
temp2 = temp[temp.isWeekend == True]
t2 = temp2['quantity'].sum() 

In [ ]:
#for weekdays
temp2 = temp[temp.date.isin(holidaysn) == False]
t2 = temp2['quantity'].sum() 

In [ ]:
temp2

In [ ]:
import statistics 
#calculate monthly average demand for 10 years
total_demand_by_week = [0,0,0,0,0,0,0]

# """Friday  Monday Saturday Sunday    Thursday Tuesday    Wednesday

days = [0,1,2,3,4,5,6]
for j in range(len(days)):
    #for j in weekly_summary[i].index:
        
    for i in range(len(years)):
            
        total_demand_by_week[j] += weekly_summary[i][j]
        
print(total_demand_by_week)

normalized = []
for i in range(len(total_demand_by_week)):
    t1 = total_demand_by_week[i] - statistics.mean(total_demand_by_week)
    if t1>0 : 
        t = t1/statistics.stdev(total_demand_by_week)
    else :
        t = (statistics.mean(total_demand_by_week) - total_demand_by_week[i] )/statistics.stdev(total_demand_by_week)
    print(t)
    normalized.append(t)
    
ax = sns.barplot(x=weekly_summary[i].index, y=normalized)
ax.set_title('Weekly Demand')
ax.set_xlabel("days")
ax.set_ylabel("avg demand")

print(max(total_demand_by_week)-min(total_demand_by_week))


In [ ]:
#calculate monthly average demand for 10 years
total_demand_by_week = [0,0,0,0,0,0,0]

# """Friday  Monday Saturday Sunday    Thursday Tuesday    Wednesday

days = [0,1,2,3,4,5,6]
for j in range(len(days)):
    #for j in weekly_summary[i].index:
        
    for i in range(len(years)):
            
        total_demand_by_week[j] += weekly_summary[i][j]

for i in range(len(total_demand_by_week)):
    total_demand_by_week[i] = total_demand_by_week[i]/len(years)
    
ax = sns.barplot(x=weekly_summary[i].index, y=total_demand_by_week)
ax.set_title('Holidays Demand')
ax.set_xlabel("days")
ax.set_ylabel("avg demand")


In [ ]:
len(holiday_summary[0])

In [ ]:
len(workday_summary[0])

In [ ]:
len(weekend_summary[0])

In [ ]:
weekend_summary